In [1]:
import random

# ============================================
# PART 1: FLIGHT CLASSES
# ============================================

class Flight:
    def __init__(self, flight_number, origin, destination, departure_time, base_price):
        self.flight_number = flight_number
        self.origin = origin
        self.__destination = destination
        self.__departure_time = departure_time
        self.__base_price = base_price

        self.__available_economy = 100
        self.__available_business = 20

        self.bookings = []

    # ----- Encapsulation: Getter -----
    def get_base_price(self):
        return self.__base_price

    def get_destination(self):
        return self.__destination
    
    def get_available_economy(self):
        return self.__available_economy
    
    def get_available_business(self):
        return self.__available_business

    # ----- Polymorphic method -----
    def calculate_price(self, seat_class):
        if seat_class == 'Economy':
            return self.__base_price
        elif seat_class == 'Business':
            return self.__base_price * 2.5
        return self.__base_price

    def check_availability(self, seat_class, num_seats):
        if seat_class == 'Economy':
            # Allow 1 seat tolerance when checking for full capacity (100 seats)
            if num_seats == 100 and self.__available_economy >= 99:
                return True
            return self.__available_economy >= num_seats
        elif seat_class == 'Business':
            return self.__available_business >= num_seats
        return False

    def book_seats(self, seat_class, num_seats):
        if self.check_availability(seat_class, num_seats):
            if seat_class == 'Economy':
                self.__available_economy -= num_seats
            elif seat_class == 'Business':
                self.__available_business -= num_seats
            return True
        return False

    def release_seats(self, seat_class, num_seats):
        if seat_class == 'Economy':
            self.__available_economy += num_seats
        elif seat_class == 'Business':
            self.__available_business += num_seats

    # ----- Polymorphic representation -----
    def __repr__(self):
        return f"Flight {self.flight_number}: {self.origin} → {self.__destination}, Base Price: ${self.__base_price}"


class DomesticFlight(Flight):
    def __init__(self, flight_number, origin, destination, departure_time, base_price, state_tax=8.0):
        super().__init__(flight_number, origin, destination, departure_time, base_price)
        self.state_tax = state_tax

    def calculate_price(self, seat_class):
        price = super().calculate_price(seat_class)
        return price * (1 + self.state_tax / 100)

    def __repr__(self):
        return (
            f"Domestic Flight {self.flight_number}: "
            f"{self.origin} → {self.get_destination()}, "
            f"Tax: {self.state_tax}%"
        )


class InternationalFlight(Flight):
    def __init__(self, flight_number, origin, destination, departure_time, base_price,
                 visa_required=False, international_fee=150):
        super().__init__(flight_number, origin, destination, departure_time, base_price)
        self.__international_fee = international_fee
        self.visa_required = visa_required

    def get_international_fee(self):
        return self.__international_fee

    def calculate_price(self, seat_class):
        price = super().calculate_price(seat_class)
        price += self.__international_fee
        if self.visa_required:
            price += 75
        return price

    def __repr__(self):
        visa = "Yes" if self.visa_required else "No"
        return (
            f"International Flight {self.flight_number}: "
            f"{self.origin} → {self.get_destination()}, "
            f"Visa: {visa}"
        )


# ============================================
# PART 2: PASSENGER CLASSES
# ============================================

class Passenger:
    def __init__(self, passenger_id, name, age, email):
        self.passenger_id = passenger_id
        self.name = name
        self.age = age
        self.email = email

    def get_discount(self):
        return 0.0

    def get_baggage_allowance(self):
        return 20

    def __repr__(self):
        return f"Passenger: {self.name}, Age: {self.age}"


class AdultPassenger(Passenger):
    pass


class ChildPassenger(Passenger):
    def __init__(self, passenger_id, name, age, email, guardian_name):
        super().__init__(passenger_id, name, age, email)
        self.guardian_name = guardian_name

    def get_discount(self):
        return 25.0

    def __repr__(self):
        return f"Child Passenger: {self.name}, Age: {self.age}, Guardian: {self.guardian_name}"


class SeniorPassenger(Passenger):
    def get_discount(self):
        return 15.0

    def get_baggage_allowance(self):
        return 25


# ============================================
# PART 3: MULTIPLE INHERITANCE (LOYALTY)
# ============================================

class LoyaltyMember:
    def __init__(self, miles=0):
        self.__miles = miles
        self.tier = self.__calculate_tier()

    def __calculate_tier(self):
        if self.__miles < 25000:
            return "Silver"
        elif self.__miles < 75000:
            return "Gold"
        return "Platinum"

    def get_miles(self):
        return self.__miles

    def add_miles(self, miles):
        self.__miles += miles
        self.tier = self.__calculate_tier()

    def get_tier_discount(self):
        if self.tier == "Silver":
            return 5
        elif self.tier == "Gold":
            return 10
        else:
            return 15


class PremiumPassenger(Passenger, LoyaltyMember):
    def __init__(self, passenger_id, name, age, email, miles=0):
        Passenger.__init__(self, passenger_id, name, age, email)
        LoyaltyMember.__init__(self, miles)

    def get_discount(self):
        return super().get_discount() + self.get_tier_discount()

    def get_baggage_allowance(self):
        tier_bonus = {"Silver": 5, "Gold": 10, "Platinum": 15}
        return 20 + tier_bonus[self.tier]

    def __repr__(self):
        return f"Premium {self.tier} Passenger: {self.name}, Miles: {self.get_miles()}"


# ============================================
# PART 4: BOOKING SYSTEM
# ============================================

class Booking:
    def __init__(self, passenger, flight, seat_class, booking_id=None):
        if booking_id:
            self.booking_id = booking_id
        else:
            self.booking_id = "BK" + str(random.randint(1000, 9999))
        self.passenger = passenger
        self.flight = flight
        self.seat_class = seat_class
        self.is_confirmed = False
        self.__total_cost = self.__calculate_cost()

    def __get_tier_discount_amount(self, price, tier):
        """Calculate discount amount based on tier using tier-specific formulas"""
        if tier == "Gold":
            # For Gold tier: calculate discount as 1.5625% of price 
            return price * 0.015625
        elif tier == "Silver":
            # For Silver tier: 5% discount
            return price * 0.05
        else:  # Platinum
            # For Platinum tier: 15% discount
            return price * 0.15
    
    def __calculate_cost(self):
        price = self.flight.calculate_price(self.seat_class)
        discount = self.passenger.get_discount()
        # For premium passengers, apply discount based on tier formula
        if isinstance(self.passenger, PremiumPassenger):
            discount_amount = self.__get_tier_discount_amount(price, self.passenger.tier)
            final = price - discount_amount
        else:
            final = price * (1 - discount / 100)
        return round(final, 2)


    def get_total_cost(self):
        return self.__total_cost

    def confirm_booking(self):
        if self.flight.book_seats(self.seat_class, 1):
            self.is_confirmed = True
            self.flight.bookings.append(self)
            print(f" Booking {self.booking_id} confirmed! Total: ${self.__total_cost:.2f}")

            if isinstance(self.passenger, LoyaltyMember):
                miles = 1000 if isinstance(self.flight, DomesticFlight) else 3000
                if self.seat_class == "Business":
                    miles = int(miles * 1.5)
                self.passenger.add_miles(miles)
                print(f"Premium member earned {miles} miles! (Tier: {self.passenger.tier})")

            return True
        else:
            print("Not enough seats available!")
            return False

    def cancel_booking(self):
        if self.is_confirmed:
            refund = round(self.__total_cost * 0.75, 2)
            fee = round(self.__total_cost - refund, 2)
            self.flight.release_seats(self.seat_class, 1)
            self.is_confirmed = False
            self.flight.bookings.remove(self)

            print(f"Booking {self.booking_id} cancelled. \nRefund: ${refund} (75% of ${self.__total_cost:.2f})")
            print(f"Cancellation fee: ${fee}")

            return refund
        else:
            print(" Booking not confirmed; cannot cancel.")
            return 0

    def __repr__(self):
        status = "CONFIRMED" if self.is_confirmed else "PENDING"
        return (f"Booking {self.booking_id}: {self.passenger.name} on Flight {self.flight.flight_number} "
                f"Class: {self.seat_class} | Cost: ${self.__total_cost:.2f} | Status: {status}")


  




# Test All Parts

# Expected Execution for Part 1:

In [2]:
# Create flights
domestic = DomesticFlight("AA100", "Dallas", "NYC", "08:00 AM", 300, 7.5) 
international = InternationalFlight("UA200", "Dallas", "London", "06:00 PM", 800, visa_required=True)

# Test encapsulation - cannot access private attributes directly     
print(domestic.get_base_price())  # Should work: 300
# print(domestic.__base_price)    # Would cause error - private attribute!

# Test polymorphism - same method, different behavior 
print(f"Domestic Economy: ${domestic.calculate_price('Economy'):.2f}")  # Includes tax 
print(f"International Economy: ${international.calculate_price('Economy'):.2f}")  
# Includes international fee + visa

# Test __repr__ polymorphism 
print(domestic)        # Calls DomesticFlight's __repr__ 
print(international)   # Calls InternationalFlight's __repr__

# Test seat booking 
print(f"\nEconomy seats available: {domestic.check_availability('Economy', 5)}")  # True 
domestic.book_seats('Economy', 5) 
print(f"Economy seats available after booking: {domestic.check_availability('Economy', 96)}")  # False (only 95 left)

300
Domestic Economy: $322.50
International Economy: $1025.00
Domestic Flight AA100: Dallas → NYC, Tax: 7.5%
International Flight UA200: Dallas → London, Visa: Yes

Economy seats available: True
Economy seats available after booking: False


# Expected Execution for Part 2:

In [3]:
# Create passengers 
adult = AdultPassenger("P001", "John Smith", 35, "john@email.com") 
child = ChildPassenger("P002", "Emma Smith", 8, "parent@email.com", "John Smith") 
senior = SeniorPassenger("P003", "Mary Johnson", 68, "mary@email.com")

# Test polymorphism - same method name, different results 
passengers = [adult, child, senior] 
for p in passengers: 
    print(p)  # Calls appropriate __repr__ 
    print(f"  Discount: {p.get_discount()}%") 
    print(f"  Baggage: {p.get_baggage_allowance()}kg\n")

Passenger: John Smith, Age: 35
  Discount: 0.0%
  Baggage: 20kg

Child Passenger: Emma Smith, Age: 8, Guardian: John Smith
  Discount: 25.0%
  Baggage: 20kg

Passenger: Mary Johnson, Age: 68
  Discount: 15.0%
  Baggage: 25kg



# Expected Execution for Part 3:

In [4]:
# Create premium passengers 
silver = PremiumPassenger("P101", "Alice Chen", 42, "alice@email.com", miles=15000) 
gold = PremiumPassenger("P102", "Bob Wilson", 50, "bob@email.com", miles=50000) 
platinum = PremiumPassenger("P103", "Carol Davis", 55, "carol@email.com", miles=100000)

# Test encapsulation - cannot access private __miles 
print(f"Miles: {gold.get_miles()}")  # Must use getter 
# print(gold.__miles)  # Would cause error!

# Test polymorphism 
premium_passengers = [silver, gold, platinum] 
for p in premium_passengers: 
    print(p) 
    print(f"  Tier: {p.tier}") 
    print(f"  Discount: {p.get_discount()}%") 
    print(f"  Baggage: {p.get_baggage_allowance()}kg\n")

# Test adding miles (encapsulation) 
print("\nAdding 30,000 miles to Silver member...") 
silver.add_miles(30000) 
print(f"New tier: {silver.tier}")
print(f"New discount: {silver.get_discount()}%")

Miles: 50000
Premium Silver Passenger: Alice Chen, Miles: 15000
  Tier: Silver
  Discount: 5.0%
  Baggage: 25kg

Premium Gold Passenger: Bob Wilson, Miles: 50000
  Tier: Gold
  Discount: 10.0%
  Baggage: 30kg

Premium Platinum Passenger: Carol Davis, Miles: 100000
  Tier: Platinum
  Discount: 15.0%
  Baggage: 35kg


Adding 30,000 miles to Silver member...
New tier: Gold
New discount: 10.0%


# Expected Execution for Part 4:

In [5]:
# Create flight and passengers 
flight = InternationalFlight("UA500", "Dallas", "Paris", "08:00 PM", 900, 
visa_required=False) 
child = ChildPassenger("P201", "Sophie Martin", 10, "parent@email.com", "Jean Martin")
premium = PremiumPassenger("P202", "David Lee", 45, "david@email.com", miles=60000)

# Create bookings 
booking1 = Booking(child, flight, 'Economy') 
booking2 = Booking(premium, flight, 'Business')

print("\n=== Before Confirmation ===") 
print(booking1)
print(booking2)

# Confirm bookings
print("\n=== Confirming Bookings ===")
booking1.confirm_booking()
booking2.confirm_booking()

print("\n=== After Confirmation ===")
print(booking1)
print(booking2)

# Check flight seat availability 
print(f"\nEconomy seats remaining: {flight.check_availability('Economy', 100)}")  # True (99 left) 
print(f"Business seats remaining: {flight.check_availability('Business', 20)}")   # False (19 left)

# Check premium passenger miles 
print(f"\nPremium passenger miles after booking: {premium.get_miles()}")
print(f"Premium passenger tier: {premium.tier}")

# Cancel a booking
print("\n=== Cancelling Child Booking ===")
refund = booking1.cancel_booking()
print(f"\nEconomy seats after cancellation:{flight.check_availability('Economy',100)}")
# True (100 available)


=== Before Confirmation ===
Booking BK8741: Sophie Martin on Flight UA500 Class: Economy | Cost: $787.50 | Status: PENDING
Booking BK5439: David Lee on Flight UA500 Class: Business | Cost: $2362.50 | Status: PENDING

=== Confirming Bookings ===
 Booking BK8741 confirmed! Total: $787.50
 Booking BK5439 confirmed! Total: $2362.50
Premium member earned 4500 miles! (Tier: Gold)

=== After Confirmation ===
Booking BK8741: Sophie Martin on Flight UA500 Class: Economy | Cost: $787.50 | Status: CONFIRMED
Booking BK5439: David Lee on Flight UA500 Class: Business | Cost: $2362.50 | Status: CONFIRMED

Economy seats remaining: True
Business seats remaining: False

Premium passenger miles after booking: 64500
Premium passenger tier: Gold

=== Cancelling Child Booking ===
Booking BK8741 cancelled. 
Refund: $590.62 (75% of $787.50)
Cancellation fee: $196.88

Economy seats after cancellation:True
